In [10]:
import pickle
import os
from joblib import load
import pandas as pd

In [24]:
base_dir = os.path.abspath("..")  # Subir un nivel desde notebooks/
path_encodings_layer_01 = os.path.join(base_dir, "path", "ivae_reactome", "seed_0", "encodings_layer-01_seed-00.pkl")
path_encodings_layer_04 = os.path.join(base_dir, "path", "ivae_reactome", "seed_0", "encodings_layer-04_seed-00.pkl")
path_metrics_seed = os.path.join(base_dir, "path", "ivae_reactome", "seed_0", "metrics-seed-00.pkl")


x_train_encoded_01 = load(path_encodings_layer_01)
x_train_encoded_04 = load(path_encodings_layer_04)
metrics_seed = load(path_metrics_seed)
    
print(metrics_seed)  # Ver qué contiene

   seed metric  split      score          model
0     0   loss  train  46.273651  ivae_reactome
1     0    mse  train   0.003067  ivae_reactome
2     0   loss    val  46.229282  ivae_reactome
3     0    mse    val   0.003072  ivae_reactome
4     0   loss   test  46.187099  ivae_reactome
5     0    mse   test   0.003067  ivae_reactome


In [25]:
x_train_encoded_01 = pd.DataFrame(x_train_encoded_01)
x_train_encoded_04 = pd.DataFrame(x_train_encoded_04)

In [26]:
# Crear objeto AnnData para análisis
adata = sc.AnnData(X=x_train_encoded_01)

/home/sfernandez/.local/lib/python3.10/site-packages/anndata/utils.py:307: UserWarning: X converted to numpy array with dtype object
  warnings.warn(f"{name} converted to numpy array with dtype {arr.dtype}")


In [29]:
# Elimina columnas no numéricas si existen
x_train_encoded_01 = x_train_encoded_01.select_dtypes(include=["number"])

# Si hay valores que deberían ser números pero están como strings, intenta convertirlos
x_train_encoded_01 = x_train_encoded_01.apply(pd.to_numeric, errors="coerce")

# Verifica que todo esté correcto antes de continuar
print(x_train_encoded_01.dtypes)


REACTOME_2_LTR_CIRCLE_FORMATION                                      float32
REACTOME_ABACAVIR_METABOLISM                                         float32
REACTOME_ABACAVIR_TRANSMEMBRANE_TRANSPORT                            float32
REACTOME_ABACAVIR_TRANSPORT_AND_METABOLISM                           float32
REACTOME_ABC_FAMILY_PROTEINS_MEDIATED_TRANSPORT                      float32
                                                                      ...   
REACTOME_ZBP1_DAI_MEDIATED_INDUCTION_OF_TYPE_I_IFNS                  float32
REACTOME_ZINC_EFFLUX_AND_COMPARTMENTALIZATION_BY_THE_SLC30_FAMILY    float32
REACTOME_ZINC_INFLUX_INTO_CELLS_BY_THE_SLC39_GENE_FAMILY             float32
REACTOME_ZINC_TRANSPORTERS                                           float32
seed                                                                   int64
Length: 1616, dtype: object


In [31]:
adata = sc.AnnData(X=x_train_encoded_01.values)  # Convertir a numpy array

# Aplicar clustering y reducción de dimensionalidad
sc.pp.neighbors(adata, use_rep="X", random_state=0)
sc.tl.leiden(adata, random_state=0)
sc.tl.umap(adata, random_state=0)

/home/sfernandez/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-06 10:12:57.846910: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738836777.857259 1506988 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738836777.860385 1506988 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-06 10:12:57.871133: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical oper